In [ ]:
# !pip install pycapnp
# !pip install typer
# !pip install ipywidgets

In [ ]:
import asyncio
import time

import ipywidgets as widgets
import numpy as np
import zmq.asyncio
from IPython.display import display

from deepdrrzmq.utils import zmq_util, typer_util, server_util, config_util
from deepdrrzmq.utils.zmq_util import zmq_no_linger_context
from deepdrrzmq.utils.typer_util import unwrap_typer_param
from deepdrrzmq.utils.server_util import messages, make_response, DeepDRRServerException
from deepdrrzmq.utils.config_util import config, load_config

In [ ]:
class ZMQAsyncMsgHandler:
    """
    Manage the setup and configuration of ZMQ publisher/subscriber sockets for communication over TCP. 
    It uses the asyncio variant of ZMQ for asynchronous operations.
    """
    def __init__(self, addr, rep_port, pub_port, sub_port, hwm):
        """
        Initializes the ZMQ connection handler with the address and port numbers for REP, PUB, and SUB sockets.

        :param addr: The address of the ZMQ server.
        :type addr: str
        :param rep_port: The port number for REP (request-reply) socket connections.
        :type rep_port: int
        :param pub_port: The port number for PUB (publish) socket connections.
        :type pub_port: int
        :param sub_port: The port number for SUB (subscribe) socket connections.
        :type sub_port: int
        :param hwm: The high water mark (HWM) for message buffering.
        :type hwm: int
        """
        
        self.addr = addr
        self.rep_port = rep_port
        self.pub_port = pub_port
        self.sub_port = sub_port
        self.hwm = hwm
        self.context = zmq.asyncio.Context() # zmq.Context()
        self.setup()

    def setup(self):
        """
        Sets up ZMQ PUB and SUB sockets for communication.
        """        
        try:
            # Setup the PUB socket
            self.pub_socket = self.context.socket(zmq.PUB)
            self.pub_socket.hwm = self.hwm
            self.pub_socket.connect(f"tcp://{self.addr}:{self.pub_port}")
            # Setup the SUB socket
            self.sub_socket = self.context.socket(zmq.SUB)
            self.sub_socket.hwm = self.hwm
            self.sub_socket.connect(f"tcp://{self.addr}:{self.sub_port}")
            self.sub_socket.subscribe(b"/loggerd/")
        except zmq.ZMQError as e:
            print(f"Failed to setup ZMQ: {str(e)}")


In [ ]:
class LoggerControllerInterface:
    def __init__(self, ZMQAMH_instance):
        self.ZMQAMH_instance = ZMQAMH_instance
        self.create_widgets()
        self.bind_event_handlers()
        self.setup_layout()

    def create_widgets(self):
        # logger widgets
        self.record_button = widgets.Button(description='New Session', disabled=True, icon='circle')
        self.stop_button = widgets.Button(description='Stop', disabled=True, icon='square')
        self.session_id = widgets.Text(value='None', description='Session ID:', disabled=True)
        self.record_status = widgets.Text(value='None', description='Recording Status:', disabled=True)
        # environment setting widgets
        self.update_setting = widgets.Button(description='Update Setting', disabled=False, icon='check', button_style='', tooltip='Click me')
        ## patient model widgets
        self.color_toggle = widgets.ToggleButtons(options=[('Opaque', 0), ('Transparent' ,1)], 
                                                 description='Patient Material:', disabled=False, button_style='')
        self.flip_patient = widgets.ToggleButtons(options=[('Original',True), ('Flipped',False)], 
                                                 description='Flip Patient:', disabled=False, button_style='')
        ## view indicator widgets
        self.view_indicator = widgets.ToggleButtons(options=[('On',True), ('Off',False)], 
                                                   description='View Indicator ON/OFF:', disabled=False, button_style='')
        self.self_view_indicator = widgets.ToggleButtons(options=[('On',True), ('Off',False)],     
                                                       description='View Indicator Self-Select ON/OFF:', disabled=False, button_style='')
        self.view_indicator_select = widgets.SelectMultiple(options=['Anteroposterior_View', 'Lateral_View', 'Inlet_View', 'Outlet_View', 'Obturator_Oblique_Left/Iliac_Oblique_Right', 'Obturator_Oblique_Right/Iliac_Oblique_Left', 'Obturator_Oblique_Inlet_Left/Iliac_Oblique_Inlet_Right','Obturator_Oblique_Outlet_Left/Iliac_Oblique_Outlet_Right','Obturator_Oblique_Inlet_Right/Iliac_Oblique_Inlet_Left','Obturator_Oblique_Outlet_Right/Iliac_Oblique_Outlet_Left', 'Teardrop_Left_View','Teardrop_Right_View','None'], 
                                                          value=['Lateral_View'], description='View Indicator Selection:', disabled=False)
        ## surgical instrument widgets
        self.kwire_indicator = widgets.ToggleButtons(options=[('On',True), ('Off',False)], 
                                                    description='Kwire Error Indicator ON/OFF:', disabled=False, button_style='')
        self.kwire_error_selection = widgets.ToggleButtons(options=[('On',True), ('Off',False)], 
                                                         description='Kwire Error Indicator Self-Selection:', disabled=False, button_style='')
        self.corridor_selection = widgets.Select(options=[('Left S1', 0), ('Left Ramus', 1), ('L Ramus Short', 2),('L Teardrop', 3),('Right S1', 4), ('Right Ramus', 5), ('Right Ramus Short', 6),('Right Teardrop', 7),('S 2', 8),('S 3', 9),('None',100)], 
                                                value=2, description='Corridor:')
        self.patient_switch= widgets.Select(options=[('100114', "100114"), ('100129', "100129"), ('100139', "100139"), ('100155', "100155"), ('100229', "100229")],
                                        value="100114",description='Patient Switch:',)

    def bind_event_handlers(self):
        self.update_setting.on_click(lambda b: asyncio.create_task(self.updatesetting_clicked(b)))
        self.record_button.on_click(lambda b: asyncio.create_task(self.on_record_button_clicked(b)))
        self.record_button.on_click(lambda b: asyncio.create_task(self.updatesetting_clicked(b)))
        self.stop_button.on_click(lambda b: asyncio.create_task(self.on_stop_button_clicked(b)))
        
    def setup_layout(self):
        # environment setting layout
        patient_model_layout = widgets.VBox([self.color_toggle, self.flip_patient, self.patient_switch])
        view_indicator_layout = widgets.VBox([self.view_indicator, self.self_view_indicator, self.view_indicator_select])
        surgical_instrument_layout = widgets.VBox([self.kwire_indicator, self.kwire_error_selection, self.corridor_selection])
        environment_setting_layout = widgets.HBox([patient_model_layout, view_indicator_layout , surgical_instrument_layout, self.update_setting])
        display(environment_setting_layout)
        # logger layout
        logger_layout = widgets.HBox([self.record_button, self.stop_button, self.session_id, self.record_status])
        display(logger_layout)
    
    async def ui_update_loop(self):
        while True:
            # print(f"enter while")
            try:
                # print(f"enter try")
                topic, data = await asyncio.wait_for(
                    self.ZMQAMH_instance.sub_socket.recv_multipart(), 
                    timeout=3.0
                )
                
                if topic == b"/loggerd/status/":
                    # print(f"enter loggerd/status/")
                    self.record_button.disabled = False
                    self.stop_button.disabled = False
                    with messages.LoggerStatus.from_bytes(data) as msg:
                        self.record_status.value = str(msg.recording)
                        self.session_id.value = msg.sessionId if msg.recording else "no session"
                        self.record_button.button_style = 'danger' if msg.recording else ''
            except asyncio.TimeoutError:
                print(f"enter asyncio timeout error")
                self.record_button.disabled = True
                self.stop_button.disabled = True
                self.session_id.value = "Disconnected"
                self.record_status.value = "Disconnected"
            except zmq.ZMQError as e:
                print(f"ZMQ Error: {e}")
            # print(f"exit try except")

            await asyncio.sleep(0.1)

    async def on_record_button_clicked(self, b):
        await self.ZMQAMH_instance.pub_socket.send_multipart([b"/loggerd/start/", b""])
        print(f"Record button clicked, b = {b}")

    async def on_stop_button_clicked(self, b):
        await self.ZMQAMH_instance.pub_socket.send_multipart([b"/loggerd/stop/", b""])
        print(f"Stop button clicked, b = {b}")
        
    async def updatesetting_clicked(self, b):
        response_topic = "/mp/setting/webgui/SettingManager/"
        msg = messages.SyncedSetting.new_message()
        msg.timestamp = msg.timestamp = float(np.float64(time.time()).item())
        msg.clientId = 'webgui'
        uiControlVar = msg.setting.init("uiControl")
        uiControlVar.patientMaterial = self.color_toggle.value
        uiControlVar.flippatient = self.flip_patient.value    
        uiControlVar.carmIndicator = self.view_indicator.value
        uiControlVar.viewIndicatorselfselect = self.self_view_indicator.value
        uiControlVar.annotationError = self.view_indicator_select.value
        uiControlVar.corridorIndicator = self.kwire_indicator.value
        uiControlVar.webcorridorerrorselect = self.kwire_error_selection.value
        uiControlVar.webcorridorselection = self.corridor_selection.value
        uiControlVar.patientCaseId= self.patient_switch.value
        await self.ZMQAMH_instance.pub_socket.send_multipart([response_topic.encode(), msg.to_bytes()])
        print(f"Update Setting button clicked, b = {b}")


In [ ]:
async def main():
    config_network = config['network']
    addr = config_network['addr_localhost']
    rep_port = config_network['rep_port']
    pub_port = config_network['pub_port']
    sub_port = config_network['sub_port']
    hwm = config_network['hwm']
    
    ZMQAMH = ZMQAsyncMsgHandler(addr, rep_port, pub_port, sub_port, hwm)
    LCI = LoggerControllerInterface(ZMQAMH)

    try:
        ui_update_task.cancel()
    except NameError:
        pass

    ui_update_task = asyncio.create_task(LCI.ui_update_loop())

if __name__ == "__main__":
    await main()